# 🗺️ station
DB에 저장할 형식의 버스와 지하철역 데이터 만들기

### TODO
- [X] 버스 route 데이터를 막차 시간과 join
- [ ] route에 포함된 버스역만 추려내기
- [ ] 모든 역데이터에 geohash 적용
- [ ] 각 역별 막차시간 생성

### 0. import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
import koreanize_matplotlib
import folium
from folium.plugins import MarkerCluster
import json

### 1. 버스 막차데이터 결측값 제거

In [2]:
bus_station = pd.read_csv('../bus/result_bus_station.csv')
train_station = pd.read_csv("../train/result_train_station_withcode.csv")
bus_route_per_station = pd.read_csv("bus_route_per_station.csv")
lastbus = pd.read_csv('../bus/bus_route_last_time.csv')

In [3]:
lastbus

,Unnamed: 0,station,arsId,stationNm,busRouteAbrv,busRouteId,busRouteNm,lastTm,dis,speed,sec
0,0,101000308,02920,동대문디자인프라자,TOUR11,100000017,TOUR11,17:10,0.0,0,0
1,0,101000309,02921,방산.중부시장,TOUR11,100000017,TOUR11,17:10,1697.0,109,57
2,0,101000310,02922,을지로3가,TOUR11,100000017,TOUR11,17:12,634.0,20,115
3,0,101000311,02923,을지로입구,TOUR11,100000017,TOUR11,17:15,1003.0,22,165
4,0,100000400,01918,청와대,TOUR11,100000017,TOUR11,17:19,2655.0,37,259
...,...,...,...,...,...,...,...,...,...,...,...
28253,0,232000569,35519,대우아파트,김포16A,241461015,김포16A,22:21,118.0,19,23
28254,0,232000295,35585,대우아파트후문,김포16A,241461015,김포16A,22:22,124.0,16,28
28255,0,232000291,35576,월드메르디앙아파트,김포16A,241461015,김포16A,22:23,519.0,20,94
28256,0,232000856,35803,청도아파트.은행정입구,김포16A,241461015,김포16A,22:24,203.0,13,57


In [4]:
lastbus = lastbus.drop(columns=["Unnamed: 0", "arsId", "dis", "speed", "sec"])
lastbus = lastbus.rename(
    columns={"station": "정류장번호", "stationNm": "정류소명", "busRouteAbrv": "노선명", "busRouteId": "노선번호", "lastTm": "막차시간"})

lastbus

,정류장번호,정류소명,노선명,노선번호,busRouteNm,막차시간
0,101000308,동대문디자인프라자,TOUR11,100000017,TOUR11,17:10
1,101000309,방산.중부시장,TOUR11,100000017,TOUR11,17:10
2,101000310,을지로3가,TOUR11,100000017,TOUR11,17:12
3,101000311,을지로입구,TOUR11,100000017,TOUR11,17:15
4,100000400,청와대,TOUR11,100000017,TOUR11,17:19
...,...,...,...,...,...,...
28253,232000569,대우아파트,김포16A,241461015,김포16A,22:21
28254,232000295,대우아파트후문,김포16A,241461015,김포16A,22:22
28255,232000291,월드메르디앙아파트,김포16A,241461015,김포16A,22:23
28256,232000856,청도아파트.은행정입구,김포16A,241461015,김포16A,22:24


In [6]:
lastbus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28258 entries, 0 to 28257
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   정류장번호       28258 non-null  int64 
 1   정류소명        28258 non-null  object
 2   노선명         28258 non-null  object
 3   노선번호        28258 non-null  int64 
 4   busRouteNm  28258 non-null  object
 5   막차시간        28258 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.3+ MB


In [8]:
lastbus[lastbus["막차시간"].str.len() != 5]

,정류장번호,정류소명,노선명,노선번호,busRouteNm,막차시간


In [9]:
lastbus["막차시간"] = lastbus["막차시간"].str.slice_replace(start=2, stop=3, repl='')

lastbus

,정류장번호,정류소명,노선명,노선번호,busRouteNm,막차시간
0,101000308,동대문디자인프라자,TOUR11,100000017,TOUR11,1710
1,101000309,방산.중부시장,TOUR11,100000017,TOUR11,1710
2,101000310,을지로3가,TOUR11,100000017,TOUR11,1712
3,101000311,을지로입구,TOUR11,100000017,TOUR11,1715
4,100000400,청와대,TOUR11,100000017,TOUR11,1719
...,...,...,...,...,...,...
28253,232000569,대우아파트,김포16A,241461015,김포16A,2221
28254,232000295,대우아파트후문,김포16A,241461015,김포16A,2222
28255,232000291,월드메르디앙아파트,김포16A,241461015,김포16A,2223
28256,232000856,청도아파트.은행정입구,김포16A,241461015,김포16A,2224


In [10]:
lastbus.astype({"막차시간": int})

,정류장번호,정류소명,노선명,노선번호,busRouteNm,막차시간
0,101000308,동대문디자인프라자,TOUR11,100000017,TOUR11,1710
1,101000309,방산.중부시장,TOUR11,100000017,TOUR11,1710
2,101000310,을지로3가,TOUR11,100000017,TOUR11,1712
3,101000311,을지로입구,TOUR11,100000017,TOUR11,1715
4,100000400,청와대,TOUR11,100000017,TOUR11,1719
...,...,...,...,...,...,...
28253,232000569,대우아파트,김포16A,241461015,김포16A,2221
28254,232000295,대우아파트후문,김포16A,241461015,김포16A,2222
28255,232000291,월드메르디앙아파트,김포16A,241461015,김포16A,2223
28256,232000856,청도아파트.은행정입구,김포16A,241461015,김포16A,2224


In [11]:
bus_route_per_station = bus_route_per_station.rename(
    columns={"ROUTE_ID": "노선번호", "NODE_ID": "정류장번호", "X좌표": "경도", "Y좌표": "위도"})

bus_route_per_station

,노선번호,노선명,순번,정류장번호,ARS_ID,정류소명,경도,위도
0,100100585,N37,32,100000001,1001,종로2가사거리,126.987750,37.569765
1,123000010,741,77,100000001,1001,종로2가사거리,126.987750,37.569765
2,100100073,470,66,100000001,1001,종로2가사거리,126.987750,37.569765
3,100100036,171,11,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183
4,100100034,162,14,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183
...,...,...,...,...,...,...,...,...
42060,100100415,6004,1,161000612,92702,인천공항T2-B1층,126.434018,37.466505
42061,100100417,6008,36,161000612,92702,인천공항T2-B1층,126.434018,37.466505
42062,100100418,6011,32,161000612,92702,인천공항T2-B1층,126.434018,37.466505
42063,100100419,6012,31,161000612,92702,인천공항T2-B1층,126.434018,37.466505


In [12]:
result_bus_time = pd.merge(lastbus, bus_route_per_station, on=['노선번호', '정류장번호'], how='left')

result_bus_time

,정류장번호,정류소명_x,노선명_x,노선번호,busRouteNm,막차시간,노선명_y,순번,ARS_ID,정류소명_y,경도,위도
0,101000308,동대문디자인프라자,TOUR11,100000017,TOUR11,1710,NaN,NaN,NaN,NaN,NaN,NaN
1,101000309,방산.중부시장,TOUR11,100000017,TOUR11,1710,NaN,NaN,NaN,NaN,NaN,NaN
2,101000310,을지로3가,TOUR11,100000017,TOUR11,1712,NaN,NaN,NaN,NaN,NaN,NaN
3,101000311,을지로입구,TOUR11,100000017,TOUR11,1715,NaN,NaN,NaN,NaN,NaN,NaN
4,100000400,청와대,TOUR11,100000017,TOUR11,1719,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
28608,232000569,대우아파트,김포16A,241461015,김포16A,2221,NaN,NaN,NaN,NaN,NaN,NaN
28609,232000295,대우아파트후문,김포16A,241461015,김포16A,2222,NaN,NaN,NaN,NaN,NaN,NaN
28610,232000291,월드메르디앙아파트,김포16A,241461015,김포16A,2223,NaN,NaN,NaN,NaN,NaN,NaN
28611,232000856,청도아파트.은행정입구,김포16A,241461015,김포16A,2224,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
result_bus_time[result_bus_time["경도"].isnull()]

,정류장번호,정류소명_x,노선명_x,노선번호,busRouteNm,막차시간,노선명_y,순번,ARS_ID,정류소명_y,경도,위도
0,101000308,동대문디자인프라자,TOUR11,100000017,TOUR11,1710,NaN,NaN,NaN,NaN,NaN,NaN
1,101000309,방산.중부시장,TOUR11,100000017,TOUR11,1710,NaN,NaN,NaN,NaN,NaN,NaN
2,101000310,을지로3가,TOUR11,100000017,TOUR11,1712,NaN,NaN,NaN,NaN,NaN,NaN
3,101000311,을지로입구,TOUR11,100000017,TOUR11,1715,NaN,NaN,NaN,NaN,NaN,NaN
4,100000400,청와대,TOUR11,100000017,TOUR11,1719,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
28608,232000569,대우아파트,김포16A,241461015,김포16A,2221,NaN,NaN,NaN,NaN,NaN,NaN
28609,232000295,대우아파트후문,김포16A,241461015,김포16A,2222,NaN,NaN,NaN,NaN,NaN,NaN
28610,232000291,월드메르디앙아파트,김포16A,241461015,김포16A,2223,NaN,NaN,NaN,NaN,NaN,NaN
28611,232000856,청도아파트.은행정입구,김포16A,241461015,김포16A,2224,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
set(result_bus_time["노선명_x"][result_bus_time["경도"].isnull()])

{'013-1',
 '013-2',
 '022B',
 '075A',
 '075A향',
 '075B',
 '077B',
 '1',
 '1-4',
 '1-8',
 '1004',
 '1006',
 '102',
 '105',
 '105-1',
 '1100',
 '1101',
 '1101N',
 '1102',
 '111',
 '1112',
 '1113-10',
 '1113-11',
 '13',
 '13-2',
 '1301',
 '1309',
 '1310',
 '1312',
 '1330-3',
 '1336',
 '1403',
 '1552',
 '1560',
 '16-1',
 '1650',
 '166-1',
 '1670-1',
 '1680',
 '169',
 '17',
 '1801',
 '2',
 '20',
 '200',
 '2000',
 '21',
 '22',
 '23',
 '231',
 '27',
 '271',
 '30',
 '300',
 '3000예',
 '3005',
 '3006',
 '3007',
 '3008',
 '301',
 '31',
 '3100',
 '3100N',
 '3102',
 '320',
 '3201',
 '3217',
 '33',
 '33-1',
 '3300',
 '3301',
 '3302',
 '331',
 '3323',
 '3400',
 '3401',
 '341',
 '3413',
 '342',
 '343',
 '35',
 '3500',
 '357',
 '360',
 '3600',
 '3700',
 '38',
 '3800',
 '4101',
 '4103',
 '4108',
 '4300',
 '4300-1',
 '4300-2',
 '4403',
 '4432',
 '452',
 '50',
 '500-1A',
 '500-2N',
 '5000A',
 '5007',
 '52',
 '5200',
 '5200-1',
 '5300-1',
 '56',
 '56-1',
 '56-2',
 '5602',
 '5609',
 '57-1',
 '5700A',
 '59',

In [15]:
set(result_bus_time["노선명_x"][result_bus_time["경도"].notnull()])

{'0411',
 '100',
 '109',
 '111',
 '1111',
 '1116',
 '1128',
 '1167',
 '120',
 '173',
 '2227',
 '2236',
 '2312',
 '241',
 '2416',
 '270',
 '271',
 '3217',
 '3314',
 '3323',
 '3413',
 '3417',
 '342',
 '3422',
 '343',
 '4312',
 '4432',
 '4435',
 '452',
 '507',
 '5511',
 '5528',
 '5616',
 '6001',
 '6002',
 '6003',
 '6004',
 '6006',
 '6007',
 '6008',
 '6009',
 '601',
 '6010',
 '6011',
 '6012',
 '6013',
 '6014',
 '6015',
 '6016',
 '6017',
 '6018',
 '6019',
 '602',
 '6020',
 '603',
 '6100',
 '6101',
 '6102',
 '6103',
 '6105',
 '6200',
 '6300',
 '654',
 '660',
 '6615',
 '6624',
 '6629',
 '6630',
 '6645',
 '6648',
 '6649',
 '6701',
 '6702',
 '6703',
 '6705',
 '6716',
 '674',
 '701',
 '705',
 '708',
 '741',
 '742',
 '761',
 '7612',
 '7734',
 '7737',
 '7739',
 '774',
 '8001',
 '8002',
 '8003',
 '8101',
 '8146',
 '8221',
 '8441',
 '8551',
 '8552',
 '8561',
 '8701',
 '8762',
 '8771',
 '8773',
 '9409',
 'N15',
 'N31',
 'N51',
 'N6001',
 'N6002',
 'N64',
 'N72',
 'N73',
 'N75',
 '강남05',
 '강남06',
 '강남

In [16]:
result_bus_time = result_bus_time[result_bus_time["경도"].notnull()]

result_bus_time

,정류장번호,정류소명_x,노선명_x,노선번호,busRouteNm,막차시간,노선명_y,순번,ARS_ID,정류소명_y,경도,위도
28,100000419,경복궁역(효자로입구),청와A01,100000020,청와대A01(자율주행),1645,청와대A01(자율주행),1.0,1280.0,경복궁역(효자로입구),126.974399,37.576579
29,100000415,국립고궁박물관,청와A01,100000020,청와대A01(자율주행),1648,청와대A01(자율주행),2.0,1119.0,국립고궁박물관,126.974046,37.578797
30,100000416,청와대,청와A01,100000020,청와대A01(자율주행),1649,청와대A01(자율주행),3.0,1601.0,청와대,126.973778,37.582747
31,100000417,춘추문,청와A01,100000020,청와대A01(자율주행),1650,청와대A01(자율주행),4.0,1602.0,춘추문,126.979731,37.582911
32,100000418,경복궁.국립민속박물관,청와A01,100000020,청와대A01(자율주행),1650,청와대A01(자율주행),5.0,1603.0,경복궁.국립민속박물관,126.979500,37.579532
...,...,...,...,...,...,...,...,...,...,...,...,...
10974,124000457,강동리버스트7.6단지,강동02,124900003,강동02,0051,강동02,73.0,25375.0,강동리버스트7.6단지,127.172374,37.570459
10975,124000458,강동리버스트상가,강동02,124900003,강동02,0052,강동02,74.0,25377.0,강동리버스트상가,127.173789,37.571724
10976,124900130,강동리버스트8단지,강동02,124900003,강동02,0053,강동02,75.0,25763.0,강동리버스트8단지,127.176109,37.572596
10977,124900128,강빛초.중학교,강동02,124900003,강동02,0056,강동02,76.0,25764.0,강빛초.중학교,127.172385,37.574029


In [17]:
result_bus_time = result_bus_time.sort_values(by=["노선번호", "순번"])
result_bus_time = result_bus_time.drop(
    columns=["ARS_ID", "busRouteNm", "정류소명_x", "노선명_x"])
result_bus_time = result_bus_time.rename(columns={"정류소명_y": "정류소명", "노선명_y": "노선명"})
result_bus_time = result_bus_time.reset_index(drop=True)

result_bus_time

,정류장번호,노선번호,막차시간,노선명,순번,정류소명,경도,위도
0,100000419,100000020,1645,청와대A01(자율주행),1.0,경복궁역(효자로입구),126.974399,37.576579
1,100000415,100000020,1648,청와대A01(자율주행),2.0,국립고궁박물관,126.974046,37.578797
2,100000416,100000020,1649,청와대A01(자율주행),3.0,청와대,126.973778,37.582747
3,100000417,100000020,1650,청와대A01(자율주행),4.0,춘추문,126.979731,37.582911
4,100000418,100000020,1650,청와대A01(자율주행),5.0,경복궁.국립민속박물관,126.979500,37.579532
...,...,...,...,...,...,...,...,...
10484,124000457,124900003,0051,강동02,73.0,강동리버스트7.6단지,127.172374,37.570459
10485,124000458,124900003,0052,강동02,74.0,강동리버스트상가,127.173789,37.571724
10486,124900130,124900003,0053,강동02,75.0,강동리버스트8단지,127.176109,37.572596
10487,124900128,124900003,0056,강동02,76.0,강빛초.중학교,127.172385,37.574029


In [18]:
result_bus_time.to_csv('result_bus_per_station.csv',
                      index=False, encoding="utf-8-sig");